# NDT Global - Technical Assessment Machine Learning Engineer ( Ex 2)

Hands on exercise - Classical Machine Learning

Human Activity Recognition database built from the recordings of 30 subjects performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors.


1 WALKING
2 WALKING_UPSTAIRS
3 WALKING_DOWNSTAIRS
4 SITTING
5 STANDING
6 LAYING


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import seaborn as sns


### Load data

In [13]:
train_features_path = 'HumanActivityData/UCI_HAR_Dataset/train/X_train.txt'
train_labels_path = 'HumanActivityData/UCI_HAR_Dataset/train/y_train.txt'
test_features_path = 'HumanActivityData/UCI_HAR_Dataset/test/X_test.txt'
test_labels_path = 'HumanActivityData/UCI_HAR_Dataset/test/y_test.txt'
features_info_path = 'HumanActivityData/UCI_HAR_Dataset/features.txt'
activity_labels_path = 'HumanActivityData/UCI_HAR_Dataset/activity_labels.txt'

In [14]:
feature_names = pd.read_csv(features_info_path, delim_whitespace=True, header=None, names=['index', 'feature'])

if feature_names['feature'].duplicated().any():
    # Append index to duplicate feature names to make them unique
    feature_names['feature'] = feature_names['feature'] + '_' + feature_names.groupby('feature').cumcount().astype(str)
    
    
X_train = pd.read_csv(train_features_path, delim_whitespace=True, header=None, names=feature_names['feature'].values)
X_test = pd.read_csv(test_features_path, delim_whitespace=True, header=None, names=feature_names['feature'].values)
y_train = pd.read_csv(train_labels_path, delim_whitespace=True, header=None, names=['label'])
y_test = pd.read_csv(test_labels_path, delim_whitespace=True, header=None, names=['label'])

In [18]:
X_train = pd.read_csv(train_features_path)
y_train = pd.read_csv(train_labels_path, header=None)

X_test = pd.read_csv(test_features_path)
y_test = pd.read_csv(test_labels_path, header=None)

In [19]:
# Load labels (space-delimited text files)
y_train = pd.read_csv(train_labels_path, delim_whitespace=True, header=None, names=['label'])
y_test = pd.read_csv(test_labels_path, delim_whitespace=True, header=None, names=['label'])


### Data preprocesing

In [20]:
# Normalize the features (Standard Scaling)
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)


ValueError: could not convert string to float: '  2.7841883e-001 -1.6410568e-002 -1.2352019e-001 -9.9824528e-001 -9.7530022e-001 -9.6032199e-001 -9.9880719e-001 -9.7491437e-001 -9.5768622e-001 -9.4306751e-001 -5.5785126e-001 -8.1840869e-001  8.4930787e-001  6.8584458e-001  8.2263681e-001 -9.8193011e-001 -9.9999130e-001 -9.9978838e-001 -9.9840537e-001 -9.9915036e-001 -9.7786550e-001 -9.4822478e-001 -7.1489166e-001 -5.0093000e-001 -5.7097906e-001  6.1162716e-001 -3.2954862e-001  2.8421321e-001  2.8459454e-001  1.1570542e-001 -9.0962529e-002  2.9431041e-001 -2.8121057e-001  8.5988430e-002 -2.2152694e-002 -1.6656535e-002 -2.2064350e-001 -1.3428663e-002 -7.2691890e-002  5.7938169e-001  9.6656113e-001 -1.4155127e-001  1.0937881e-001 -9.9741134e-001 -9.8944741e-001 -9.3163868e-001 -9.9788359e-001 -9.8961366e-001 -9.3324040e-001  8.9206031e-001 -1.6134256e-001  1.2258573e-001  9.8452014e-001 -1.1489334e-001  1.0276411e-001 -3.8342955e-001  9.0782890e-001 -9.7058275e-001 -9.7850045e-001 -9.9918

In [25]:
# walking vs others
# Convert labels: 1 = walking, 0 = other activities
y_train_binary = y_train['label'].apply(lambda x: 1 if x == 1 else 0)
y_test_binary = y_test['label'].apply(lambda x: 1 if x == 1 else 0)

### ML Models

Let's try a random forest. Other options could be a SVM or regression.

In [26]:
rf_binary = RandomForestClassifier(n_estimators=100, random_state=21)
rf_binary.fit(X_train_scaled, y_train_binary)

NameError: name 'X_train_scaled' is not defined

In [27]:
y_pred_binary = rf_binary.predict(X_test_scaled)


NameError: name 'X_test_scaled' is not defined

i've run out of time :(